In [4]:
import robin_stocks as r
import sys
import config
import datetime as dt
import time
import pyotp
import pandas as pd
import numpy as np
import yfinance as y
import ta
import warnings
import pytz
# import ta
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
import mibian

In [94]:
# Login
login = loginer()

In [21]:
starting = '1900-01-01'
ending = '2025-01-01'

In [79]:
# Functions
def get_stocks():
    stocks = list()
    stocks.append('PLTR')
    stocks.append('NIO')
    stocks.append('AAPL')
#     stocks.append('TSLA')
    return (stocks)

def quote(ticker):
    ro = r.get_latest_price(ticker)
    print(ticker.upper() + ": $" + str(ro[0]))

def buy_exact(ticker, amount):
    ro = r.order_buy_market(ticker, amount)
    print(ro)

def sell_exact(ticker, amount):
    ro = r.order_sell_market(ticker, amount)
    print(ro)

def open_market():
    market = False
    tz = pytz.timezone('America/Los_Angeles')
    time_now = dt.datetime.now(tz).time()
    
    market_open = dt.time(6,0,0)
    market_close = dt.time(13,29,0)
    
    if time_now > market_open and time_now < market_close:
        market = True
    else:
        pass
    return market

def latest_price(name):
    return r.stocks.get_latest_price(name)[0]

# Returns a dataframe of options available from the start day to
# end day
def options_within_days(options, start, end):
    tz = pytz.timezone('America/Los_Angeles')
    time_now = dt.datetime.now(tz).time()
    start_date = (dt.datetime.combine(dt.date(2021,1,1),time_now)+dt.timedelta(start))
    end_date = (dt.datetime.combine(dt.date(2021,1,1),time_now)+dt.timedelta(end))
    dates = [start_date + dt.timedelta(days=x) for x in range(0, (end_date-start_date).days)]
    list_of_dates = []
    for d in dates:
        list_of_dates.append(d.strftime("%Y-%m-%d"))
    new_df = df[df['expiration_date'].isin(list_of_dates)].reset_index(drop=True)
    return new_df.sort_values(by='expiration_date')

# Gets the options for a range of days, start from 0 if you want the immediate result
# result. You have to pass the number of days in Integer
def get_option(name, start=0, end=1000):
    curr = []
    for i in name:
        data = r.options.find_tradable_options(name)
        df = pd.DataFrame(data)
        curr.append(options_within_days(df, start, end))
    return pd.concat(curr, axis=1, keys=name)

# Returns the most available detialed information of the last 59 days in
# a dataframe. It takes a list of inputs
def history_last_sixty(inp):
    current_frames = []
    for i in inp:
        tick = y.Ticker(i)
        df = tick.history(period='59d', interval='15m')
        current_frames.append(df)
    return pd.concat(current_frames, axis=1, keys=inp)

# Information about the stocks specific group
def stock_info(inp, start= starting, end=ending, grp ='Open'):
    if grp == 'Open':
        return y.download(inp, start=start, end=end).Open
    elif grp == 'High':
        return y.download(inp, start=start, end=end).High
    elif grp == 'Low':
        return y.download(inp, start=start, end=end).Low
    elif grp == 'Close':
        return y.download(inp, start=start, end=end).Close
    elif grp == 'Volume':
        return y.download(inp, start=start, end=end).Volume


# Returns a dataframe of all the stocks history from yahoo finance API
# It takes a start and end date which lets you select the interval you want
# the information from
# Its result are daily and the prices shown are during the market hours
def history(inp,start='1800-01-01',end='2800-01-01'):
    return y.download(inp, start=start, end=end, group_by='tickers')

# Returns the day where the result was maximized
def max_date(inp, start=starting, end=ending, grp='Open'):
    ans = stock_info(inp, start, end, grp)
    ret_list = []
    for i in inp:
        if len(inp)==1:
            date=ans.idxmax()
            ret_list.append(date)
            return ret_list
        date = ans[i].idxmax()
        ret_list.append(date)
    return ret_list

In [76]:
# Euro Vanilla call for stocks that don't offer dividends
# Euro Vanilla is tested but Black Scholes has not been tested
# Returns a percentage that this stock might likely profit, the default return value is call profit
def euro_vanilla(spot_price, strike_price, time_to_expire, interest_rate, sigma, option='call'):
    share1 = (np.log(spot_price/strike_price) + (interest_rate + 0.5 * sigma ** 2) * time_to_expire) / (sigma * np.sqrt(time_to_expire))
    share2 = (np.log(spot_price/strike_price) + (interest_rate - 0.5 * sigma ** 2) * time_to_expire) / (sigma * np.sqrt(time_to_expire))
    
    if option == 'call':
        ans = (spot_price * si.norm.cdf(share1, 0.0, 1.0) -strike_price *np.exp(-interest_rate * time_to_expire) * si.norm.cdf(share2))
    else:
        ans = (strike_price * np.exp(-interest_rate * time_to_expire) * si.norm.cdf(-share2, 0.0, 1.0) - spot_price * si.norm.cdf(-share1, 0.0,1.0))
    return ans

# Black Scholes Call profit calculation 
def black_scholes(spot_price, strike_price, time_to_expire, interest_rate, rate_dividend, sigma, option='call'):
    share1 = (np.log(spot_price/strike_price)+(interest_rate - rate_dividend + 0.5 * sigma ** 2) * time_to_expire) / (sigma * np.sqrt(time_to_expire))
    share2 = (np.log(spot_price/strike_price)+(interest_rate - rate_dividend - 0.5 * sigma ** 2) * time_to_expire) / (sigma * np.sqrt(time_to_expire))
    
    if option == 'call':
        ans = (spot_price * np.exp(-rate_dividend * time_to_expire) * si.norm.cdf(share1,0.0,1.0) - strike_price * np.exp(-interest_rate * time_to_expire) *si.norm.cdf(share2, 0.0, 1.0))
    else:
        ans = (strike_price * np.exp(-interest_rate * time_to_expire) * si.norm.cdf(-share2,0.0,1.0) - spot_price * np.exp(-rate_dividend * time_to_expire) *si.norm.cdf(-share1, 0.0, 1.0))
    return ans

In [90]:
# plotting
# Most functions needs documentation    

# Plots and normalize the data for multiple stocks
# For a stock to be included, the starting date must be available/the same
def multi_plotter(inp, start= starting, end=ending, grp='Open'):
    curr = stock_info(inp, start, end, grp)
    # Normalizes the plots so they start from the same spot
    fig = (curr/curr.iloc[0]*100).plot(figsize=(20,10))
    plt.legend()
    return fig

# Mulit_plotter but without normalization of data
def multi_plotter_none(inp, start= starting, end=ending, grp='Open'):
    curr = stock_info(inp, start, end, grp)
    for i in inp:
        i.upper()
        curr[i].plot(label=str(i), figsize=(20,10))
    plt.legend()

# Plots the daily return
def daily_ret(inp, start=starting, end = ending):
    ret_list = []
    for i in inp:
        i.upper()
        tick_hist = history(i, start, end);
        daily_return = (tick_hist['Adj Close'] / tick_hist['Adj Close'].shift(1)) -1
        ret_list.append(daily_return)
    return ret_list

# Plots the daily difference and daily return plots (Need Confirmation)
def daily_plotter(inp, start=starting, end = ending):
    curr = daily_ret(inp, start, end)
    for i in range(len(inp)):
        price_diff_plot(curr[i], inp[i])
        daily_ret_plot(curr[i], inp[i])

# This function will plot the daily price difference and gives you the option to select
# which max or min result would you want from the group type of the stocks passed in
# It will then plot everything 
def daily_plotter_max_min(inp, start=starting, end=ending, grp='Open', mode='Max'):
    curr = daily_ret(inp, start, end)
    if mode == "Max":
        date = max_date(inp, start,end,grp)
    elif mode == "Min":
        date = min_date(inp, start, end,grp)
    else:
        return error
    for i in range(len(inp)):
        current = curr[i].to_frame()
        current.columns = ['Simple daily Return']
        current['Simple daily Return'].plot()
        plt.xlabel("Date")
        plt.ylabel("Percent of Change")
        plt.title(inp[i])
        plt.axvline(date[i], color='red')
        plt.show()

def price_diff_plot(df, inp):
    df = df.to_frame()
    df.columns = ['Simple daily Return']
    df['Simple daily Return'].plot()
    plt.xlabel("Date")
    plt.ylabel("Percent")
    plt.title(inp)
    plt.show()
    plt.clf()
    
# Plots a histogram of 60 bins of daily return
def daily_ret_plot(df,inp, start=starting, end=ending):
    daily = daily_ret(inp, start, end)
    df = df.to_frame()
    df.columns = ['Simple daily Return']
    df.plot.hist(bins=60)
    plt.xlabel("Daily Return in %")
    plt.ylabel("Percent")
    plt.title(inp)
    plt.show()
    plt.clf()
    

# A simple moving average calculator and plotter
def roll_plotter(inp, start=starting, end=ending, roll1=20, roll2=100, grp='Open'):
    for i in inp:
        i = i.upper()
        curr = stock_info(get_stocks(), start=start, end=end, grp='Close')
        current = curr.loc[:,i]
        short = current.rolling(window=roll1).mean()
        long = current.rolling(window=roll2).mean()
        fig, ax=plt.subplots(figsize=(16,9))
        ax.plot(current.index, current, label=i)
        ax.plot(short.index, short, label=str(roll1)+ ' days of rolling')
        ax.plot(long.index, long, label=str(roll2)+ ' days of rolling')
        ax.set_xlabel('date')
        ax.set_ylabel('adjusted closing price ($)')
        ax.legend()
        plt.show()
        plt.clf()
        

In [87]:
# history_last_sixty(get_stocks())
# history(get_stocks(),start='2020-05-05')
# daily_plotter(get_stocks(), start = '2020-10-12')
# daily_ret(get_stocks(), start='2004-01-01')
# get_option(get_stocks()) #Requires login
# multi_plotter(get_stocks(), start='2020-11-10', grp='Close') # options are Close, Open, High, Low, Volume
# roll_plotter(get_stocks(), start='2020-01-10', grp='Volume')
daily_plotter_max_min(get_stocks(), start='2020-08-10',grp='Volume', mode='Min')
# multi_plotter_none(get_stocks(), grp='Close', start= '2020-10-10')

Resources Used:
    <ol> <li> https://aaronschlegel.me/black-scholes-formula-python.html</li>
    <li> https://alexandrenesovic.com/2019/11/17/how-to-find-import-and-plot-historical-financial-data-with-yfinance-python-and-how-to-calculate-the-daily-returns/ </li>
    <li> https://www.learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/ </li>
    </ol>
    